In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="17rFuCNZUUY1xHrMq1WTamV-JWh_IDZe8", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/04_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why It Matters Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_02_why_it_matters_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Equivalence
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_03_math_equivalence.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Data Model Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_04_data_model_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# From NCSN to Diffusion Models -- The Unified View -- Vizuara

## 1. Why Does This Matter?

In the previous notebooks, we built Noise Conditioned Score Networks from scratch and used Annealed Langevin Dynamics to generate samples. But there is a bigger picture here.

In 2020, Ho, Jain, and Abbeel published DDPM (Denoising Diffusion Probabilistic Models). At first glance, DDPM looks very different from NCSN. DDPM adds noise in discrete time steps and learns to reverse the process. NCSN uses multiple noise levels and predicts score functions.

But here is the remarkable insight: **these are the same thing in disguise**. The score of the noisy distribution is directly proportional to the noise that was added. Learning to predict noise (DDPM) and learning to predict the score (NCSN) are equivalent tasks.

By the end of this notebook, you will:
- See the mathematical equivalence between NCSN and DDPM objectives
- Implement both perspectives on the same data
- Understand how the Score SDE framework unifies them
- Build intuition for why modern diffusion models are all score-based

## 2. Building Intuition

Imagine two people looking at the same mountain from different sides.

**Person A (NCSN perspective):** "I am going to learn the slope of the mountain at every point. If I know the slope everywhere, I can start from any random location and follow the slopes downhill to find the valleys where the treasure is."

**Person B (DDPM perspective):** "I am going to learn to undo erosion. Given any eroded version of the mountain, I can predict what it looked like one step before the erosion happened. By repeatedly undoing erosion, I can reconstruct the original mountain."

Both are describing the same underlying process. The "slope at a point" (score) and "the erosion that happened" (noise) are directly related:

$$\text{score} = -\frac{\text{noise}}{\text{noise level}}$$

This connection is what Yang Song and colleagues formalized in 2021 using Stochastic Differential Equations (SDEs).

## 3. The Mathematics

### 3.1 The NCSN Objective (Score Prediction)

$$\mathcal{L}_{\text{NCSN}} = \mathbb{E}_\sigma \mathbb{E}_{x, \epsilon} \left[ \sigma^2 \left\| s_\theta(\tilde{x}, \sigma) + \frac{\epsilon}{\sigma} \right\|^2 \right]$$

Target: predict the score $-\epsilon / \sigma$

### 3.2 The DDPM Objective (Noise Prediction)

$$\mathcal{L}_{\text{DDPM}} = \mathbb{E}_{t, x, \epsilon} \left[ \left\| \epsilon_\theta(\tilde{x}, t) - \epsilon \right\|^2 \right]$$

Target: predict the noise $\epsilon$

### 3.3 The Equivalence

If $s_\theta(\tilde{x}, \sigma) = -\epsilon_\theta(\tilde{x}, \sigma) / \sigma$, then:

$$\sigma^2 \left\| s_\theta + \frac{\epsilon}{\sigma} \right\|^2 = \sigma^2 \left\| -\frac{\epsilon_\theta}{\sigma} + \frac{\epsilon}{\sigma} \right\|^2 = \left\| \epsilon_\theta - \epsilon \right\|^2$$

**Worked example:** Let $\epsilon = [0.5, -0.3]$, $\sigma = 2.0$.

NCSN target: $-\epsilon / \sigma = [-0.25, 0.15]$

Suppose score network predicts $s = [-0.2, 0.12]$:
- NCSN loss: $\sigma^2 \|[-0.2, 0.12] - [-0.25, 0.15]\|^2 = 4 \cdot ((0.05)^2 + (0.03)^2) = 4 \cdot 0.0034 = 0.0136$

Corresponding noise prediction: $\epsilon_\theta = -\sigma \cdot s = -2.0 \cdot [-0.2, 0.12] = [0.4, -0.24]$:
- DDPM loss: $\|[0.4, -0.24] - [0.5, -0.3]\|^2 = (-0.1)^2 + (0.06)^2 = 0.01 + 0.0036 = 0.0136$

The losses are identical. This is exactly what we want.

## 4. Let's Build It -- Component by Component

### 4.1 Setup: Data and NCSN from Previous Notebooks

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

def generate_data(n=5000):
    mix = torch.rand(n, 1)
    centers = torch.tensor([[-3.0, 0.0], [3.0, 0.0]])
    idx = (mix > 0.5).long().squeeze()
    return centers[idx] + 0.5 * torch.randn(n, 2)

class ScoreNet(nn.Module):
    """NCSN: predicts score s(x, sigma)."""
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 2),
        )
    def forward(self, x, sigma):
        if sigma.dim() == 1:
            sigma = sigma.unsqueeze(1)
        return self.net(torch.cat([x, sigma], dim=1))

class NoiseNet(nn.Module):
    """DDPM-style: predicts noise epsilon(x, sigma)."""
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim), nn.SiLU(),
            nn.Linear(hidden_dim, 2),
        )
    def forward(self, x, sigma):
        if sigma.dim() == 1:
            sigma = sigma.unsqueeze(1)
        return self.net(torch.cat([x, sigma], dim=1))

data = generate_data(5000)
L = 10
sigma_1, sigma_L = 10.0, 0.01
sigmas = torch.tensor([sigma_1 * (sigma_L / sigma_1) ** (i / (L-1)) for i in range(L)])
print("Data and models ready.")

In [ ]:
#@title 🎧 Transition: Train Side By Side Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_05_train_side_by_side_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Train Both Networks Side by Side

In [ ]:
#@title 🎧 Code Walkthrough: Train Side By Side Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_06_train_side_by_side_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Train NCSN (score prediction)
score_model = ScoreNet(128)
score_opt = torch.optim.Adam(score_model.parameters(), lr=1e-3)

# Train NoiseNet (noise prediction, DDPM-style)
noise_model = NoiseNet(128)
noise_opt = torch.optim.Adam(noise_model.parameters(), lr=1e-3)

score_losses = []
noise_losses = []

for epoch in range(3000):
    idx = torch.randint(0, L, (data.shape[0],))
    sigma = sigmas[idx].unsqueeze(1)
    epsilon = torch.randn_like(data)
    noisy_data = data + sigma * epsilon

    # --- NCSN loss ---
    score_target = -epsilon / sigma
    score_pred = score_model(noisy_data, sigma.squeeze(1))
    loss_score = (sigma**2 * (score_pred - score_target)**2).mean()

    score_opt.zero_grad()
    loss_score.backward()
    score_opt.step()
    score_losses.append(loss_score.item())

    # --- DDPM loss ---
    noise_pred = noise_model(noisy_data, sigma.squeeze(1))
    loss_noise = ((noise_pred - epsilon)**2).mean()

    noise_opt.zero_grad()
    loss_noise.backward()
    noise_opt.step()
    noise_losses.append(loss_noise.item())

    if (epoch + 1) % 500 == 0:
        print(f"Epoch {epoch+1}: Score loss={loss_score.item():.4f}, Noise loss={loss_noise.item():.4f}")

In [ ]:
#@title 🎧 What to Look For: Plot Training Curves
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_07_plot_training_curves.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].semilogy(score_losses, alpha=0.3, color='blue')
axes[0].set_title('NCSN (Score Prediction) Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True, alpha=0.3)

axes[1].semilogy(noise_losses, alpha=0.3, color='red')
axes[1].set_title('DDPM-style (Noise Prediction) Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Verify Equivalence Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_08_verify_equivalence_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Verify the Equivalence Numerically

In [ ]:
#@title 🎧 Code Walkthrough: Verify Equivalence Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_09_verify_equivalence_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Verify Equivalence Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_10_verify_equivalence_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Show that score = -noise_pred / sigma
test_points = torch.randn(100, 2) * 3
test_sigma_val = 1.0
test_sigma = torch.full((100,), test_sigma_val)

with torch.no_grad():
    score_output = score_model(test_points, test_sigma)
    noise_output = noise_model(test_points, test_sigma)

# If equivalence holds: score should approximately equal -noise / sigma
derived_score = -noise_output / test_sigma_val

# They won't be exactly equal (different random seeds, different networks)
# but they should be correlated and similar in magnitude
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(score_output[:, 0].numpy(), derived_score[:, 0].numpy(), s=10, alpha=0.5)
axes[0].plot([-5, 5], [-5, 5], 'r--', label='Perfect agreement')
axes[0].set_title('Score (x1 component): Direct vs Derived from Noise')
axes[0].set_xlabel('NCSN score prediction')
axes[0].set_ylabel('-NoiseNet / sigma')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].scatter(score_output[:, 1].numpy(), derived_score[:, 1].numpy(), s=10, alpha=0.5)
axes[1].plot([-5, 5], [-5, 5], 'r--', label='Perfect agreement')
axes[1].set_title('Score (x2 component): Direct vs Derived from Noise')
axes[1].set_xlabel('NCSN score prediction')
axes[1].set_ylabel('-NoiseNet / sigma')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Both models learn the same underlying function, just parameterized differently.")
print("The correlation shows the mathematical equivalence in action.")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_11_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** The scatter plots should show a clear positive correlation along the diagonal line.

## 5. Your Turn

### TODO 1: Sample from the Noise-Prediction Model

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_12_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def sample_ddpm_style(noise_model, sigmas, n_steps_per_level=100,
                       eps=0.00005, dim=2):
    """
    Generate samples using the noise-prediction model with ALD.

    The key insight: we can convert noise predictions to scores on-the-fly.
    score = -noise_pred / sigma

    Then use the standard ALD update:
    x_{t+1} = x_t + alpha * score + sqrt(2*alpha) * z

    Args:
        noise_model: trained NoiseNet
        sigmas: noise levels (large to small)
        n_steps_per_level: steps per noise level
        eps: base step size
        dim: data dimensionality

    Returns:
        Final sample tensor
    """
    # YOUR CODE HERE
    # Hint: it is almost identical to annealed_langevin_dynamics
    # but replace model(x, sigma) with -noise_model(x, sigma) / sigma
    pass

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_13_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Compare Samples from Both Models

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_14_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def compare_models(score_model, noise_model, sigmas, data, n_samples=200):
    """
    Generate samples from both models and compare them.

    For each model:
        1. Generate n_samples using ALD
        2. Plot the generated samples
        3. Compute histogram overlap with true data

    Display side by side.
    """
    # YOUR CODE HERE
    pass

In [ ]:
#@title 🎧 Transition: Putting It Together Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_15_putting_it_together_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
#@title 🎧 Code Walkthrough: Generate Samples Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_16_generate_samples_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Generate samples from both models
def ald_sample(model, sigmas, n_steps=100, eps=5e-5, is_noise_model=False):
    """Unified sampling function for both score and noise models."""
    x = torch.randn(1, 2) * sigmas[0].item()
    sigma_L = sigmas[-1].item()

    for sigma_val in sigmas.numpy():
        alpha = eps * (sigma_val / sigma_L) ** 2
        for _ in range(n_steps):
            sigma_t = torch.full((1,), sigma_val)
            with torch.no_grad():
                pred = model(x, sigma_t)
                if is_noise_model:
                    score = -pred / sigma_val
                else:
                    score = pred
            x = x + alpha * score + (2 * alpha) ** 0.5 * torch.randn_like(x)
    return x.detach()

# Generate from both
score_samples = []
noise_samples = []
for i in range(300):
    score_samples.append(ald_sample(score_model, sigmas, is_noise_model=False).numpy())
    noise_samples.append(ald_sample(noise_model, sigmas, is_noise_model=True).numpy())
    if (i+1) % 100 == 0:
        print(f"Generated {i+1}/300 from each model")

score_samples = np.concatenate(score_samples)
noise_samples = np.concatenate(noise_samples)

In [ ]:
#@title 🎧 What to Look For: Comparison Plot Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_17_comparison_plot_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: Comparison Plot Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_18_comparison_plot_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Comparison plot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(data[:500, 0].numpy(), data[:500, 1].numpy(), s=3, alpha=0.3, c='gray')
axes[0].set_title('True Data', fontsize=13)

axes[1].scatter(score_samples[:, 0], score_samples[:, 1], s=3, alpha=0.3, c='blue')
axes[1].set_title('NCSN (Score) Samples', fontsize=13)

axes[2].scatter(noise_samples[:, 0], noise_samples[:, 1], s=3, alpha=0.3, c='red')
axes[2].set_title('DDPM-style (Noise) Samples', fontsize=13)

for ax in axes:
    ax.set_xlim(-7, 7)
    ax.set_ylim(-4, 4)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle('Same Task, Different Parameterizations', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("Both models produce similar results, confirming the mathematical equivalence!")

In [ ]:
#@title 🎧 Transition: Unified View Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_19_unified_view_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

In [ ]:
#@title 🎧 Listen: Score Sde Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_20_score_sde_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# The bigger picture: Score SDE framework

print("=" * 60)
print("THE UNIFIED VIEW: Score SDEs")
print("=" * 60)
print()
print("NCSN (2019): Discrete noise levels, score prediction")
print("  - Train: s_theta(x, sigma) to predict score")
print("  - Sample: Annealed Langevin Dynamics")
print()
print("DDPM (2020): Discrete time steps, noise prediction")
print("  - Train: epsilon_theta(x, t) to predict noise")
print("  - Sample: Reverse denoising steps")
print()
print("Score SDE (2021): Continuous time, unified framework")
print("  - Forward: dx = f(x,t)dt + g(t)dw  (adds noise)")
print("  - Reverse: dx = [f - g^2 * score]dt + g * dw_rev")
print("  - Key: the reverse process depends ONLY on the score")
print()
print("This means ANY generative model that learns to denoise")
print("is implicitly learning the score function.")
print("=" * 60)

In [ ]:
#@title 🎧 What to Look For: Timeline Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_21_timeline_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the connection
fig, ax = plt.subplots(figsize=(14, 6))

# Timeline
years = [2005, 2011, 2019, 2020, 2021]
labels = ['Score\nMatching', 'Denoising\nScore Matching', 'NCSN', 'DDPM', 'Score SDE\n(Unified)']
colors = ['#4a90d9', '#4a90d9', '#2ecc71', '#e74c3c', '#9b59b6']

ax.scatter(years, [1]*len(years), s=300, c=colors, zorder=5, edgecolors='black')

for yr, label, c in zip(years, labels, colors):
    ax.annotate(label, (yr, 1), textcoords="offset points",
                xytext=(0, 30), ha='center', fontsize=11, fontweight='bold',
                color=c)
    ax.annotate(str(yr), (yr, 1), textcoords="offset points",
                xytext=(0, -25), ha='center', fontsize=10)

# Arrows
ax.annotate('', xy=(2011, 1), xytext=(2005, 1),
            arrowprops=dict(arrowstyle='->', color='gray', lw=2))
ax.annotate('', xy=(2019, 1), xytext=(2011, 1),
            arrowprops=dict(arrowstyle='->', color='gray', lw=2))
ax.annotate('', xy=(2021, 1), xytext=(2019, 1),
            arrowprops=dict(arrowstyle='->', color='#2ecc71', lw=2.5))
ax.annotate('', xy=(2021, 1), xytext=(2020, 1),
            arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=2.5))

ax.set_xlim(2003, 2023)
ax.set_ylim(0.5, 1.8)
ax.set_title('Evolution of Score-Based and Diffusion Models', fontsize=14, fontweight='bold')
ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Final Output Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_22_final_output_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Final Summary Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_23_final_summary_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Final Summary Viz Text
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_24_final_summary_viz_text.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final summary: the equivalence in one visualization

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top-left: NCSN score field
x_grid = np.linspace(-7, 7, 20)
y_grid = np.linspace(-4, 4, 15)
X, Y = np.meshgrid(x_grid, y_grid)
pts = torch.tensor(np.stack([X.ravel(), Y.ravel()], axis=1), dtype=torch.float32)
sigma_test = torch.full((pts.shape[0],), 1.0)

with torch.no_grad():
    score_field = score_model(pts, sigma_test).numpy()
    noise_field = noise_model(pts, sigma_test).numpy()

norms = np.sqrt(score_field[:, 0]**2 + score_field[:, 1]**2).reshape(15, 20)
max_n = np.percentile(norms, 95) + 1e-6

axes[0, 0].quiver(X, Y, score_field[:, 0].reshape(15,20)/max_n,
                   score_field[:, 1].reshape(15,20)/max_n, color='blue', alpha=0.7)
axes[0, 0].set_title('NCSN: Score Field at sigma=1.0', fontsize=12)
axes[0, 0].scatter(data[:200, 0].numpy(), data[:200, 1].numpy(), s=2, c='red', alpha=0.3)

derived = -noise_field / 1.0
axes[0, 1].quiver(X, Y, derived[:, 0].reshape(15,20)/max_n,
                   derived[:, 1].reshape(15,20)/max_n, color='red', alpha=0.7)
axes[0, 1].set_title('DDPM: -Noise/sigma at sigma=1.0', fontsize=12)
axes[0, 1].scatter(data[:200, 0].numpy(), data[:200, 1].numpy(), s=2, c='blue', alpha=0.3)

# Bottom: samples
axes[1, 0].scatter(score_samples[:, 0], score_samples[:, 1], s=5, c='blue', alpha=0.3)
axes[1, 0].set_title('Samples from Score Model', fontsize=12)

axes[1, 1].scatter(noise_samples[:, 0], noise_samples[:, 1], s=5, c='red', alpha=0.3)
axes[1, 1].set_title('Samples from Noise Model', fontsize=12)

for ax in axes.ravel():
    ax.set_xlim(-7, 7)
    ax.set_ylim(-4, 4)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.2)

plt.suptitle('The NCSN-DDPM Equivalence: Same Function, Different Names',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nBoth fields point in the same direction.")
print("Both sample sets match the true distribution.")
print("Score prediction and noise prediction are two views of the same coin.")
print("\nThis insight led to the unified Score SDE framework (2021),")
print("which underpins ALL modern diffusion models -- DALL-E, Stable Diffusion,")
print("Imagen, and more.")

In [ ]:
#@title 🎧 Listen: Reflection Next Steps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_25_reflection_next_steps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

**What we learned:**
1. NCSN predicts the score: $s_\theta(\tilde{x}, \sigma) \approx -\epsilon / \sigma$
2. DDPM predicts the noise: $\epsilon_\theta(\tilde{x}, t) \approx \epsilon$
3. These are mathematically equivalent: $s = -\epsilon_\theta / \sigma$
4. The Score SDE framework (2021) unified both perspectives using continuous-time SDEs

**Reflection questions:**
- If both approaches are equivalent, why did researchers develop both independently?
- What advantages might the SDE formulation have over discrete noise levels?
- Modern models like Stable Diffusion use the noise-prediction parameterization. Can you think of a practical reason why?

**Congratulations!** You now understand the complete NCSN pipeline and its connection to the broader landscape of diffusion models. The score function -- this simple compass pointing toward data -- turned out to be one of the most important ideas in modern generative AI.

**Further reading:**
- Song & Ermon, "Generative Modeling by Estimating Gradients of the Data Distribution" (NeurIPS 2019)
- Ho et al., "Denoising Diffusion Probabilistic Models" (NeurIPS 2020)
- Song et al., "Score-Based Generative Modeling through SDEs" (ICLR 2021)